**1.Seup and Imports**

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
import requests
from scipy import stats

warnings.filterwarnings('ignore')


os.makedirs('outputs', exist_ok=True)
os.makedirs('csv_files', exist_ok=True)


def setup_matplotlib_for_plotting():

    plt.switch_backend("Agg")
    plt.style.use("seaborn-v0_8")
    sns.set_palette("husl")
    plt.rcParams["font.sans-serif"] = ["Noto Sans CJK SC", "WenQuanYi Zen Hei", "PingFang SC", "Arial Unicode MS", "Hiragino Sans GB", "DejaVu Sans"]
    plt.rcParams["axes.unicode_minus"] = False
    plt.rcParams['figure.figsize'] = [12, 6]
    plt.rcParams['figure.dpi'] = 100

setup_matplotlib_for_plotting()
print("Libraries loaded successfully!")

Libraries loaded successfully!


**2.Data Loading**

In [32]:
try:
    df_trades = pd.read_csv('/content/historical_data.csv')
    print(f"Historical Trades Dataset Shape: {df_trades.shape}")
    print(f"\nColumns: {df_trades.columns.tolist()}")
except FileNotFoundError:
    print("Error: historical_data.csv file not found. Please ensure the file is in the current directory.")
    exit(1)
#Load Fear & Greed Index Data

try:
    df_sentiment = pd.read_csv('fear_greed_index.csv')
    print(f"\nSentiment Dataset Shape: {df_sentiment.shape}")
except FileNotFoundError:
    print("Fear/Greed file not found. Creating from API or sample data...")
    # Fetch from Alternative.me API (Bitcoin Fear & Greed Index)
    try:
        url = "https://api.alternative.me/fng/?limit=365&amp;format=json "
        response = requests.get(url, timeout=10)
        data = response.json()
    df_sentiment = pd.DataFrame(data['data'])
    df_sentiment['timestamp'] = pd.to_datetime(df_sentiment['timestamp'].astype(int), unit='s')
    df_sentiment['Date'] = df_sentiment['timestamp'].dt.date
    df_sentiment['value'] = df_sentiment['value'].astype(int)

    # Create Classification based on value
    def classify_sentiment(value):
        if value <= 25:
            return 'Extreme Fear'
        elif value <= 45:
            return 'Fear'
        elif value <= 55:
            return 'Neutral'
        elif value <= 75:
            return 'Greed'
        else:
            return 'Extreme Greed'

    df_sentiment['Classification'] = df_sentiment['value'].apply(classify_sentiment)
    df_sentiment = df_sentiment[['Date', 'value', 'Classification', 'value_classification']]
    df_sentiment.columns = ['Date', 'Value', 'Classification', 'Original_Classification']
    print("Fetched Fear/Greed data from API successfully!")
except Exception as e:
    print(f"API fetch failed: {e}")
    # Create sample data based on typical patterns
    dates = pd.date_range(start='2024-12-01', end='2025-04-30', freq='D')
    np.random.seed(42)
    values = np.random.randint(20, 85, size=len(dates))

    df_sentiment = pd.DataFrame({
        'Date': dates.date,
        'Value': values
    })
    df_sentiment['Classification'] = df_sentiment['Value'].apply(classify_sentiment)
    print("Created sample Fear/Greed data.")

Historical Trades Dataset Shape: (211224, 16)

Columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']

Sentiment Dataset Shape: (2644, 4)


**3.Data Preprocessing and Cleaning**

In [26]:
df_trades.columns = df_trades.columns.str.strip()
print("\nData Types:")
print(df_trades.dtypes)
print("\n" + "="*50)
print("\nMissing Values:")
print(df_trades.isnull().sum())
def parse_timestamp(ts):
    """Parse timestamp from various formats"""
    if pd.isna(ts):
        return pd.NaT
    try:
        # Try DD-MM-YYYY HH:MM format first
        return pd.to_datetime(ts, format='%d-%m-%Y %H:%M')
    except:
        try:
            return pd.to_datetime(ts)
        except:
            return pd.NaT

df_trades['Datetime'] = df_trades['Timestamp IST'].apply(parse_timestamp)
df_trades['Date'] = df_trades['Datetime'].dt.date
df_trades['Hour'] = df_trades['Datetime'].dt.hour
df_trades['DayOfWeek'] = df_trades['Datetime'].dt.dayofweek
df_trades['DayName'] = df_trades['Datetime'].dt.day_name()

print(f"\nDate Range: {df_trades['Date'].min()} to {df_trades['Date'].max()}")
print(f"Total Trading Days: {df_trades['Date'].nunique()}")
numeric_cols = ['Execution Price', 'Size Tokens', 'Size USD', 'Start Position', 'Closed PnL', 'Fee']

for col in numeric_cols:
    if col in df_trades.columns:
        df_trades[col] = pd.to_numeric(df_trades[col], errors='coerce')

df_trades['Account_Short'] = df_trades['Account'].str[:10] + '...'

print("\nNumeric columns converted successfully!")
print(df_trades[numeric_cols].describe())

df_trades['Is_Profitable'] = df_trades['Closed PnL'] > 0
df_trades['Is_Buy'] = df_trades['Side'] == 'BUY'
df_trades['Trade_Type'] = df_trades['Direction'].fillna('Unknown')
def categorize_trade_size(size_usd):
    if pd.isna(size_usd):
        return 'Unknown'
    elif size_usd < 100:
        return 'Micro (<$100)'
    elif size_usd < 1000:
        return 'Small ($100-$1K)'
    elif size_usd < 10000:
        return 'Medium ($1K-$10K)'
    elif size_usd < 100000:
        return 'Large ($10K-$100K)'
    else:
        return 'Whale (>$100K)'

df_trades['Trade_Size_Category'] = df_trades['Size USD'].apply(categorize_trade_size)

print("\nFeature Engineering Complete!")
print(f"\nTrade Size Distribution:")
print(df_trades['Trade_Size_Category'].value_counts())


Data Types:
Account              object
Coin                 object
Execution Price     float64
Size Tokens         float64
Size USD            float64
Side                 object
Timestamp IST        object
Start Position      float64
Direction            object
Closed PnL          float64
Transaction Hash     object
Order ID              int64
Crossed                bool
Fee                 float64
Trade ID            float64
Timestamp           float64
dtype: object


Missing Values:
Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

Date Range: 2023-05-01 to 2025-05-01
Total Trading Days: 480

Numeric columns converted successfully!
       Execution Price   Size Tokens   

**4. Exploratory Data Analysis (EDA)**

In [28]:
#Basic Statistics

print("\n" + "="*60)
print("TRADING DATA SUMMARY")
print("="*60)
print(f"Total Trades: {len(df_trades):,}")
print(f"Unique Traders: {df_trades['Account'].nunique():,}")
print(f"Unique Coins/Tokens: {df_trades['Coin'].nunique()}")
print(f"Trading Period: {df_trades['Date'].min()} to {df_trades['Date'].max()}")
print(f"\nTotal Volume (USD): {df_trades['Size USD'].sum():,.2f}")
print(f"Average Trade Size: {df_trades['Size USD'].mean():,.2f}")
print(f"Median Trade Size: {df_trades['Size USD'].median():,.2f}")
print(f"\nTotal PnL: {df_trades['Closed PnL'].sum():,.2f}")
print(f"Total Fees Paid: ${df_trades['Fee'].sum():,.2f}")
#Most traded coins

coin_stats = df_trades.groupby('Coin').agg({
    'Size USD': ['count', 'sum', 'mean'],
    'Closed PnL': 'sum',
    'Account': 'nunique'
}).round(2)

coin_stats.columns = ['Trade_Count', 'Total_Volume', 'Avg_Trade_Size', 'Total_PnL', 'Unique_Traders']
coin_stats = coin_stats.sort_values('Total_Volume', ascending=False)

print("\nTop 10 Most Traded Coins by Volume:")
print(coin_stats.head(10))
#Visualize Top Coins

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
#Top 10 coins by volume

top_coins = coin_stats.head(10)
axes[0].barh(top_coins.index, top_coins['Total_Volume'], color='steelblue')
axes[0].set_xlabel('Total Volume (USD)')
axes[0].set_title('Top 10 Coins by Trading Volume')
axes[0].invert_yaxis()
#Top 10 coins by trade count

top_by_count = coin_stats.sort_values('Trade_Count', ascending=False).head(10)
axes[1].barh(top_by_count.index, top_by_count['Trade_Count'], color='coral')
axes[1].set_xlabel('Number of Trades')
axes[1].set_title('Top 10 Coins by Trade Count')
axes[1].invert_yaxis()

plt.tight_layout()
plt.savefig('outputs/top_coins_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/top_coins_analysis.png")
#Trading Activity by Time

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
#Hourly distribution

hourly_trades = df_trades.groupby('Hour').size()
axes[0, 0].bar(hourly_trades.index, hourly_trades.values, color='teal')
axes[0, 0].set_xlabel('Hour of Day (IST)')
axes[0, 0].set_ylabel('Number of Trades')
axes[0, 0].set_title('Trading Activity by Hour')
axes[0, 0].set_xticks(range(0, 24))
#Day of week distribution

dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_trades = df_trades.groupby('DayName').size().reindex(dow_order)
axes[0, 1].bar(range(7), dow_trades.values, color='purple')
axes[0, 1].set_xlabel('Day of Week')
axes[0, 1].set_ylabel('Number of Trades')
axes[0, 1].set_title('Trading Activity by Day of Week')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
#Buy vs Sell distribution

side_counts = df_trades['Side'].value_counts()
axes[1, 0].pie(side_counts.values, labels=side_counts.index, autopct='%1.1f%%',
               colors=['green', 'red'], startangle=90)
axes[1, 0].set_title('Buy vs Sell Distribution')
#Trade size distribution

size_order = ['Micro (<$100)', 'Small ($100-$1K)', 'Medium ($1K-$10K)', 'Large ($10K-$100K)', 'Whale (>$100K)']
size_counts = df_trades['Trade_Size_Category'].value_counts().reindex(size_order).dropna()
axes[1, 1].bar(range(len(size_counts)), size_counts.values, color='orange')
axes[1, 1].set_xlabel('Trade Size Category')
axes[1, 1].set_ylabel('Number of Trades')
axes[1, 1].set_title('Trade Size Distribution')
axes[1, 1].set_xticks(range(len(size_counts)))
axes[1, 1].set_xticklabels(size_counts.index, rotation=45, ha='right')

plt.tight_layout()
plt.savefig('outputs/trading_activity_patterns.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/trading_activity_patterns.png")
#PnL Distribution Analysis

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
#Filter trades with non-zero PnL

pnl_trades = df_trades[df_trades['Closed PnL'] != 0]['Closed PnL']
#PnL histogram (clipped for visualization)

pnl_clipped = pnl_trades.clip(-1000, 1000)
axes[0].hist(pnl_clipped, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Break-even')
axes[0].axvline(x=pnl_trades.median(), color='green', linestyle='--', linewidth=2, label=f'Median: ${pnl_trades.median():.2f}')
axes[0].set_xlabel('Closed PnL (USD)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('PnL Distribution (Clipped to +/- $1000)')
axes[0].legend()
#Win/Loss ratio

wins = (pnl_trades > 0).sum()
losses = (pnl_trades < 0).sum()
axes[1].bar(['Winning Trades', 'Losing Trades'], [wins, losses], color=['green', 'red'])
axes[1].set_ylabel('Number of Trades')
axes[1].set_title(f'Win/Loss Distribution (Win Rate: {wins/(wins+losses)*100:.1f}%)')
#Add value labels

for i, v in enumerate([wins, losses]):
    axes[1].text(i, v + 100, f'{v:,}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/pnl_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/pnl_distribution.png")


TRADING DATA SUMMARY
Total Trades: 211,224
Unique Traders: 32
Unique Coins/Tokens: 246
Trading Period: 2023-05-01 to 2025-05-01

Total Volume (USD): 1,191,187,442.46
Average Trade Size: 5,639.45
Median Trade Size: 597.05

Total PnL: 10,296,958.94
Total Fees Paid: $245,857.72

Top 10 Most Traded Coins by Volume:
          Trade_Count  Total_Volume  Avg_Trade_Size   Total_PnL  \
Coin                                                              
BTC             26064  6.442321e+08        24717.32   868044.73   
HYPE            68005  1.419902e+08         2087.94  1948484.60   
SOL             10691  1.250748e+08        11699.07  1639555.93   
ETH             11158  1.182810e+08        10600.56  1319978.84   
@107            29992  5.576086e+07         1859.19  2783912.92   
FARTCOIN         4650  8.311390e+06         1787.40  -100687.21   
SUI              1979  7.781168e+06         3931.87   199268.83   
TRUMP            1920  7.349347e+06         3827.78  -364824.91   
MELANIA         

**5. Trader Performance Analysis**

In [29]:
#Analyze individual trader performance

trader_stats = df_trades.groupby('Account').agg({
    'Size USD': ['count', 'sum', 'mean'],
    'Closed PnL': ['sum', 'mean', 'std'],
    'Fee': 'sum',
    'Coin': 'nunique',
    'Date': 'nunique'
}).round(2)

trader_stats.columns = ['Trade_Count', 'Total_Volume', 'Avg_Trade_Size',
                        'Total_PnL', 'Avg_PnL', 'PnL_Std', 'Total_Fees',
                        'Coins_Traded', 'Active_Days']
#Calculate win rate per trader

win_rates = df_trades[df_trades['Closed PnL'] != 0].groupby('Account').apply(
    lambda x: (x['Closed PnL'] > 0).sum() / len(x) * 100
).round(2)
trader_stats['Win_Rate'] = win_rates
#Calculate net PnL (after fees)

trader_stats['Net_PnL'] = trader_stats['Total_PnL'] - trader_stats['Total_Fees']
#Sharpe-like ratio (PnL/Std)

trader_stats['Risk_Adj_Return'] = (trader_stats['Avg_PnL'] / trader_stats['PnL_Std']).replace([np.inf, -np.inf], np.nan)

print(f"\nTotal Unique Traders: {len(trader_stats)}")
print(f"\nTrader Performance Statistics:")
print(trader_stats.describe())
#Top Performers vs Bottom Performers

print("\n" + "="*60)
print("TOP 10 PROFITABLE TRADERS")
print("="*60)
top_traders = trader_stats.nlargest(10, 'Net_PnL')[['Trade_Count', 'Total_Volume', 'Net_PnL', 'Win_Rate', 'Active_Days']]
print(top_traders.to_string())

print("\n" + "="*60)
print("BOTTOM 10 TRADERS (Highest Losses)")
print("="*60)
bottom_traders = trader_stats.nsmallest(10, 'Net_PnL')[['Trade_Count', 'Total_Volume', 'Net_PnL', 'Win_Rate', 'Active_Days']]
print(bottom_traders.to_string())
#Trader Segmentation

def categorize_trader(row):
    if row['Net_PnL'] > 1000 and row['Win_Rate'] > 55:
        return 'Elite'
    elif row['Net_PnL'] > 0 and row['Win_Rate'] > 50:
        return 'Profitable'
    elif row['Net_PnL'] > -500:
        return 'Break-even'
    elif row['Win_Rate'] < 40:
        return 'Struggling'
    else:
        return 'At Risk'

trader_stats['Trader_Category'] = trader_stats.apply(categorize_trader, axis=1)

print("\nTrader Segmentation:")
trader_segment_counts = trader_stats['Trader_Category'].value_counts()
print(trader_segment_counts)
#Visualize

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
#Trader categories pie chart

colors = {'Elite': 'gold', 'Profitable': 'green', 'Break-even': 'gray', 'Struggling': 'orange', 'At Risk': 'red'}
pie_colors = [colors.get(cat, 'blue') for cat in trader_segment_counts.index]
axes[0].pie(trader_segment_counts.values, labels=trader_segment_counts.index,
            autopct='%1.1f%%', colors=pie_colors, startangle=90)
axes[0].set_title('Trader Segmentation')
#PnL vs Volume scatter

scatter = axes[1].scatter(trader_stats['Total_Volume'], trader_stats['Net_PnL'],
                          c=trader_stats['Win_Rate'], cmap='RdYlGn', alpha=0.6, s=50)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Total Volume (USD)')
axes[1].set_ylabel('Net PnL (USD)')
axes[1].set_title('Trader Performance: Volume vs PnL (Color = Win Rate)')
axes[1].set_xscale('log')
plt.colorbar(scatter, ax=axes[1], label='Win Rate %')

plt.tight_layout()
plt.savefig('outputs/trader_segmentation.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/trader_segmentation.png")



Total Unique Traders: 32

Trader Performance Statistics:
        Trade_Count  Total_Volume  Avg_Trade_Size     Total_PnL     Avg_PnL  \
count     32.000000  3.200000e+01       32.000000  3.200000e+01   32.000000   
mean    6600.750000  3.722461e+07     6007.930313  3.217800e+05   97.686250   
std     8250.373724  7.706987e+07     7281.058509  4.948276e+05  134.917766   
min      332.000000  7.280575e+05      507.630000 -1.676211e+05  -38.290000   
25%     1381.750000  4.069408e+06     2011.070000  4.689324e+04    8.872500   
50%     3699.000000  1.173684e+07     3327.970000  1.176551e+05   41.110000   
75%     8862.500000  3.529832e+07     7375.527500  4.063941e+05  148.007500   
max    40184.000000  4.208766e+08    34396.580000  2.143383e+06  520.900000   

           PnL_Std    Total_Fees  Coins_Traded  Active_Days    Win_Rate  \
count    32.000000     32.000000     32.000000     32.00000   32.000000   
mean    856.055937   7683.054688     25.031250     73.15625   85.011875   
std  

**6. Sentiment Data Integration**

In [44]:
#Prepare sentiment data for merging

# The 'Date' column is already prepared and standardized in the previous cell.
# The original line 'df_sentiment['date'] = pd.to_datetime(df_sentiment['date']).dt.date' is no longer needed.

print("\nSentiment Data Overview:")
print(f"Date Range: {df_sentiment['Date'].min()} to {df_sentiment['Date'].max()}") # Use 'Date' column
print("\nSentiment Distribution:")
print(df_sentiment['Classification'].value_counts()) # This will now work

# Ensure df_trades has the 'Date' column, in case previous cells were not run in sequence
if 'Date' not in df_trades.columns:
    def parse_timestamp(ts):
        """Parse timestamp from various formats"""
        if pd.isna(ts):
            return pd.NaT
        try:
            # Try DD-MM-YYYY HH:MM format first
            return pd.to_datetime(ts, format='%d-%m-%Y %H:%M')
        except:
            try:
                return pd.to_datetime(ts)
            except:
                return pd.NaT

    df_trades['Datetime'] = df_trades['Timestamp IST'].apply(parse_timestamp)
    df_trades['Date'] = df_trades['Datetime'].dt.date
    df_trades['Hour'] = df_trades['Datetime'].dt.hour
    df_trades['DayOfWeek'] = df_trades['Datetime'].dt.dayofweek
    df_trades['DayName'] = df_trades['Datetime'].dt.day_name()

# Ensure 'Is_Buy' column exists in df_trades before merging
df_trades['Is_Buy'] = df_trades['Side'] == 'BUY'

#Merge trading data with sentiment
df_merged = df_trades.merge(df_sentiment, on='Date', how='left') # This will now work

print(f"\nMerged Dataset Shape: {df_merged.shape}")
print(f"Trades with Sentiment Data: {df_merged['Classification'].notna().sum():,}")
print(f"Trades without Sentiment Data: {df_merged['Classification'].isna().sum():,}")
#Fill missing sentiment with 'Unknown'

df_merged['Classification'] = df_merged['Classification'].fillna('Unknown')
df_merged['Value'] = df_merged['Value'].fillna(50)  # Neutral value
#Visualize Sentiment Over Time

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(2, 1, figsize=(16, 10))
#Daily trading volume

daily_volume = df_merged.groupby('Date')['Size USD'].sum()
daily_sentiment = df_sentiment.set_index('Date')['Value'] # Use 'Date' column

ax1 = axes[0]
ax1.bar(daily_volume.index, daily_volume.values, color='steelblue', alpha=0.7, label='Daily Volume')
ax1.set_ylabel('Trading Volume (USD)', color='steelblue')
ax1.set_title('Daily Trading Volume vs Fear/Greed Index')
ax1.tick_params(axis='y', labelcolor='steelblue')

ax2 = ax1.twinx()
ax2.plot(daily_sentiment.index, daily_sentiment.values, color='red', linewidth=2, label='Fear/Greed Index')
ax2.axhline(y=50, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=25, color='green', linestyle=':', alpha=0.5, label='Extreme Fear')
ax2.axhline(y=75, color='red', linestyle=':', alpha=0.5, label='Extreme Greed')
ax2.set_ylabel('Fear/Greed Index', color='red')
ax2.tick_params(axis='y', labelcolor='red')
ax2.set_ylim(0, 100)
#Sentiment classification count

sentiment_counts = df_merged[df_merged['Classification'] != 'Unknown']['Classification'].value_counts()
colors_map = {'Extreme Fear': 'darkgreen', 'Fear': 'lightgreen', 'Neutral': 'gray',
              'Greed': 'lightsalmon', 'Extreme Greed': 'darkred'}
bar_colors = [colors_map.get(cat, 'blue') for cat in sentiment_counts.index]
axes[1].bar(sentiment_counts.index, sentiment_counts.values, color=bar_colors)
axes[1].set_xlabel('Market Sentiment')
axes[1].set_ylabel('Number of Trades')
axes[1].set_title('Trade Count by Market Sentiment')

plt.tight_layout()
plt.savefig('outputs/sentiment_overview.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/sentiment_overview.png")


Sentiment Data Overview:
Date Range: 2018-02-01 to 2025-05-02

Sentiment Distribution:
Classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64

Merged Dataset Shape: (211224, 22)
Trades with Sentiment Data: 211,218
Trades without Sentiment Data: 6
Saved: outputs/sentiment_overview.png


**7.Trader Behavior vs Market Sentiment Analysis**

In [42]:
#Analyze trading behavior during different sentiment periods

sentiment_analysis = df_merged[df_merged['Classification'] != 'Unknown'].groupby('Classification').agg({
    'Size USD': ['count', 'sum', 'mean'],
    'Closed PnL': ['sum', 'mean'],
    'Account': 'nunique',
    'Is_Buy': 'mean'  # Buy ratio
}).round(2)

sentiment_analysis.columns = ['Trade_Count', 'Total_Volume', 'Avg_Trade_Size',
                               'Total_PnL', 'Avg_PnL', 'Active_Traders', 'Buy_Ratio']

sentiment_analysis['Buy_Ratio'] = (sentiment_analysis['Buy_Ratio'] * 100).round(1)

print("\nTrading Behavior by Market Sentiment:")
print(sentiment_analysis)
#Calculate win rates by sentiment

def calc_win_rate(group):
    closed = group[group['Closed PnL'] != 0]
    if len(closed) == 0:
        return np.nan
    return (closed['Closed PnL'] > 0).mean() * 100

win_rates_by_sentiment = df_merged[df_merged['Classification'] != 'Unknown'].groupby('Classification').apply(calc_win_rate)
sentiment_analysis['Win_Rate'] = win_rates_by_sentiment.round(2)

print("\nSentiment Analysis with Win Rates:")
print(sentiment_analysis.to_string())
#Visualize Trading Behavior by Sentiment

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
analysis_ordered = sentiment_analysis.reindex(sentiment_order).dropna()
#Average Trade Size by Sentiment

colors = ['darkgreen', 'lightgreen', 'gray', 'lightsalmon', 'darkred']
axes[0, 0].bar(analysis_ordered.index, analysis_ordered['Avg_Trade_Size'], color=colors[:len(analysis_ordered)])
axes[0, 0].set_ylabel('Average Trade Size (USD)')
axes[0, 0].set_title('Average Trade Size by Market Sentiment')
axes[0, 0].tick_params(axis='x', rotation=45)
#Average PnL by Sentiment

pnl_colors = ['green' if x > 0 else 'red' for x in analysis_ordered['Avg_PnL']]
axes[0, 1].bar(analysis_ordered.index, analysis_ordered['Avg_PnL'], color=pnl_colors)
axes[0, 1].axhline(y=0, color='black', linestyle='--')
axes[0, 1].set_ylabel('Average PnL (USD)')
axes[0, 1].set_title('Average PnL by Market Sentiment')
axes[0, 1].tick_params(axis='x', rotation=45)

#Win Rate by Sentiment
axes[1, 0].bar(analysis_ordered.index, analysis_ordered['Win_Rate'], color=colors[:len(analysis_ordered)])
axes[1, 0].set_ylabel('Win Rate (%)')
axes[1, 0].set_title('Win Rate by Market Sentiment')
axes[1, 0].tick_params(axis='x', rotation=45)

#Buy Ratio by Sentiment
axes[1, 1].bar(analysis_ordered.index, analysis_ordered['Buy_Ratio'], color=colors[:len(analysis_ordered)])
axes[1, 1].set_ylabel('Buy Ratio (%)')
axes[1, 1].set_title('Buy Ratio by Market Sentiment')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('outputs/sentiment_behavior_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/sentiment_behavior_analysis.png")
#Calculate correlations

# The daily_metrics DataFrame is not defined yet. Need to create it before calculating correlations.
# Let's create a daily_metrics DataFrame with relevant aggregated data from df_merged and sentiment values.

daily_metrics = df_merged.groupby('Date').agg(
    Trade_Count=('Account', 'count'),
    Total_Volume=('Size USD', 'sum'),
    Total_PnL=('Closed PnL', 'sum'),
    Active_Traders=('Account', 'nunique'),
    Sentiment_Value=('Value', 'first') # Assuming 'Value' is the numeric sentiment score
).reset_index()

daily_metrics = daily_metrics.dropna(subset=['Sentiment_Value'])

correlations = daily_metrics.corr(numeric_only=True)['Sentiment_Value'].drop('Sentiment_Value').sort_values()

print("\nCorrelation between Fear/Greed Index and Trading Metrics:")
print("="*50)
for metric, corr in correlations.items():
    direction = "positive" if corr > 0 else "negative"
    strength = "strong" if abs(corr) > 0.5 else "moderate" if abs(corr) > 0.3 else "weak"
    print(f"{metric}: {corr:.3f} ({strength} {direction})")
#Visualize Correlations

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
#Scatter plots with regression lines

metrics_to_plot = ['Trade_Count', 'Total_Volume', 'Total_PnL', 'Active_Traders']
titles = ['Trade Count vs Sentiment', 'Volume vs Sentiment', 'PnL vs Sentiment', 'Active Traders vs Sentiment']

for idx, (metric, title) in enumerate(zip(metrics_to_plot, titles)):
    ax = axes[idx // 2, idx % 2]
    x = daily_metrics['Sentiment_Value'].dropna()
    y = daily_metrics[metric].loc[x.index]
    ax.scatter(x, y, alpha=0.6, color='steelblue')

    # Add regression line
    mask = ~(np.isnan(x) | np.isnan(y))
    if mask.sum() > 2:
        slope, intercept, r_value, p_value, std_err = stats.linregress(x[mask], y[mask])
        line_x = np.linspace(x.min(), x.max(), 100)
        line_y = slope * line_x + intercept
        ax.plot(line_x, line_y, color='red', linewidth=2, label=f'R² = {r_value**2:.3f}')

    ax.set_xlabel('Fear/Greed Index')
    ax.set_ylabel(metric.replace('_', ' '))
    ax.set_title(title)
    ax.legend()
    ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig('outputs/sentiment_correlations.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/sentiment_correlations.png")


Trading Behavior by Market Sentiment:
                Trade_Count  Total_Volume  Avg_Trade_Size   Total_PnL  \
Classification                                                          
Extreme Fear          21400  1.144843e+08         5349.73   739110.25   
Extreme Greed         39992  1.244652e+08         3112.25  2715171.31   
Fear                  61837  4.833248e+08         7816.11  3357155.44   
Greed                 50303  2.885825e+08         5736.88  2150129.27   
Neutral               37686  1.802421e+08         4782.73  1292920.68   

                Avg_PnL  Active_Traders  Buy_Ratio  
Classification                                      
Extreme Fear      34.54              32       51.0  
Extreme Greed     67.89              30       45.0  
Fear              54.29              32       49.0  
Greed             42.74              31       49.0  
Neutral           34.31              31       50.0  

Sentiment Analysis with Win Rates:
                Trade_Count  Total_Volume 

**8.Advanced Pattern Discovery**

In [45]:
# Analyze Contrarian vs Momentum Traders
# Contrarian: Buy during Fear, Sell during Greed
# Momentum: Buy during Greed, Sell during Fear

def classify_trade_style(row):
    if row['Classification'] in ['Extreme Fear', 'Fear']:
        if row['Side'] == 'BUY':
            return 'Contrarian'
        else:
            return 'Momentum'
    elif row['Classification'] in ['Extreme Greed', 'Greed']:
        if row['Side'] == 'SELL':
            return 'Contrarian'
        else:
            return 'Momentum'
    else:
        return 'Neutral'

df_merged['Trade_Style'] = df_merged.apply(classify_trade_style, axis=1)

# Ensure df_merged has necessary time-based features for analysis
# This block ensures 'Hour', 'DayOfWeek', 'DayName' are always present in df_merged
# derived from its 'Datetime' column, if not already there.
if 'Datetime' in df_merged.columns:
    if 'Hour' not in df_merged.columns:
        df_merged['Hour'] = df_merged['Datetime'].dt.hour
    if 'DayOfWeek' not in df_merged.columns:
        df_merged['DayOfWeek'] = df_merged['Datetime'].dt.dayofweek
    if 'DayName' not in df_merged.columns:
        df_merged['DayName'] = df_merged['Datetime'].dt.day_name()

# Analyze performance by trade style
style_performance = df_merged[df_merged['Trade_Style'] != 'Neutral'].groupby('Trade_Style').agg({
    'Size USD': ['count', 'mean'],
    'Closed PnL': ['sum', 'mean']
}).round(2)

style_performance.columns = ['Trade_Count', 'Avg_Size', 'Total_PnL', 'Avg_PnL']
# Win rate by style

style_win_rates = df_merged[(df_merged['Trade_Style'] != 'Neutral') & (df_merged['Closed PnL'] != 0)].groupby('Trade_Style').apply(
    lambda x: (x['Closed PnL'] > 0).mean() * 100
).round(2)
style_performance['Win_Rate'] = style_win_rates

print("\nTrading Style Performance Analysis:")
print("="*50)
print(style_performance.to_string())

print("\n" + "="*50)
print("INSIGHT: Contrarian vs Momentum Strategy Comparison")
print("="*50)
if len(style_performance) >= 2:
    contrarian_pnl = style_performance.loc['Contrarian', 'Avg_PnL'] if 'Contrarian' in style_performance.index else 0
    momentum_pnl = style_performance.loc['Momentum', 'Avg_PnL'] if 'Momentum' in style_performance.index else 0
if contrarian_pnl > momentum_pnl:
    print(f"Contrarian trading shows better average returns (${contrarian_pnl:.2f} vs ${momentum_pnl:.2f})")
else:
    print(f"Momentum trading shows better average returns (${momentum_pnl:.2f} vs ${contrarian_pnl:.2f})")
# Analyze Leverage Usage by Sentiment
# High leverage during extreme sentiment could indicate risk-taking behavior
# Calculate implied leverage from position sizes

df_merged['Implied_Leverage'] = np.where(
    df_merged['Start Position'] != 0,
    df_merged['Size USD'] / np.abs(df_merged['Start Position']),
    np.nan
)

leverage_by_sentiment = df_merged[df_merged['Classification'] != 'Unknown'].groupby('Classification').agg({
    'Implied_Leverage': ['mean', 'median', 'std']
}).round(4)

print("\nLeverage Usage by Market Sentiment:")
print(leverage_by_sentiment)
# Time-based patterns: Do profitable traders trade at specific times?

profitable_trades = df_merged[df_merged['Closed PnL'] > 0]
losing_trades = df_merged[df_merged['Closed PnL'] < 0]

setup_matplotlib_for_plotting()
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# Hourly pattern

profitable_hourly = profitable_trades.groupby('Hour').size()
losing_hourly = losing_trades.groupby('Hour').size()

x = np.arange(24)
width = 0.35

axes[0].bar(x - width/2, profitable_hourly.reindex(range(24), fill_value=0), width, label='Profitable', color='green', alpha=0.7)
axes[0].bar(x + width/2, losing_hourly.reindex(range(24), fill_value=0), width, label='Losing', color='red', alpha=0.7)
axes[0].set_xlabel('Hour of Day (IST)')
axes[0].set_ylabel('Number of Trades')
axes[0].set_title('Profitable vs Losing Trades by Hour')
axes[0].legend()
axes[0].set_xticks(range(0, 24, 2))
# Day of week pattern

profitable_dow = profitable_trades.groupby('DayOfWeek').size()
losing_dow = losing_trades.groupby('DayOfWeek').size()

x = np.arange(7)
axes[1].bar(x - width/2, profitable_dow.reindex(range(7), fill_value=0), width, label='Profitable', color='green', alpha=0.7)
axes[1].bar(x + width/2, losing_dow.reindex(range(7), fill_value=0), width, label='Losing', color='red', alpha=0.7)
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Number of Trades')
axes[1].set_title('Profitable vs Losing Trades by Day')
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[1].legend()

plt.tight_layout()
plt.savefig('outputs/time_patterns.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/time_patterns.png")
# Sentiment Regime Analysis
# Group consecutive days of same sentiment into regimes

df_sentiment_sorted = df_sentiment.sort_values('Date').copy()
df_sentiment_sorted['Regime_Change'] = (df_sentiment_sorted['Classification'] != df_sentiment_sorted['Classification'].shift()).cumsum()

regime_analysis = df_sentiment_sorted.groupby('Regime_Change').agg({
    'Date': ['min', 'max', 'count'],
    'Classification': 'first',
    'Value': 'mean'
})

regime_analysis.columns = ['Start_Date', 'End_Date', 'Duration_Days', 'Sentiment', 'Avg_Value']
regime_analysis = regime_analysis.reset_index(drop=True)

print("\nMarket Sentiment Regimes:")
print("="*60)
print(f"Total Regimes Identified: {len(regime_analysis)}")
print(f"\nAverage Regime Duration: {regime_analysis['Duration_Days'].mean():.1f} days")
print(f"\nLongest Regimes:")
print(regime_analysis.nlargest(5, 'Duration_Days')[['Start_Date', 'End_Date', 'Duration_Days', 'Sentiment']].to_string())


Trading Style Performance Analysis:
             Trade_Count  Avg_Size   Total_PnL  Avg_PnL  Win_Rate
Trade_Style                                                      
Contrarian         88984   5630.33  6370610.13    71.59     85.63
Momentum           84548   6030.27  2590956.15    30.64     81.37

INSIGHT: Contrarian vs Momentum Strategy Comparison
Contrarian trading shows better average returns ($71.59 vs $30.64)

Leverage Usage by Market Sentiment:
               Implied_Leverage                      
                           mean  median           std
Classification                                       
Extreme Fear       4.792640e+04  0.1506  6.219957e+06
Extreme Greed      1.324116e+04  0.1375  6.256774e+05
Fear               4.435088e+05  0.1789  6.266725e+07
Greed              7.905342e+05  0.2182  1.613560e+08
Neutral            2.321480e+06  0.2097  3.999713e+08
Saved: outputs/time_patterns.png

Market Sentiment Regimes:
Total Regimes Identified: 595

Average Regime Dura

**9. Key Insights & Recommendations**

In [46]:
# 9. Key Insights & Recommendations
# Generate Summary Statistics

print("\n" + "="*70)
print("                    KEY FINDINGS SUMMARY")
print("="*70)
# 1. Overall Trading Statistics

print("\n1. OVERALL TRADING STATISTICS")
print("-"*40)
print(f"   Total Trades Analyzed: {len(df_trades):,}")
print(f"   Unique Traders: {df_trades['Account'].nunique():,}")
print(f"   Total Volume: {df_trades['Size USD'].sum():,.2f}")
print(f"   Net PnL: {df_trades['Closed PnL'].sum():,.2f}")
print(f"   Overall Win Rate: {(df_trades[df_trades['Closed PnL'] != 0]['Closed PnL'] > 0).mean()*100:.1f}%")
# 2. Sentiment Impact

print("\n2. SENTIMENT IMPACT ON TRADING")
print("-"*40)
if len(sentiment_analysis) > 0:
    best_sentiment = sentiment_analysis['Avg_PnL'].idxmax()
    worst_sentiment = sentiment_analysis['Avg_PnL'].idxmin()
    print(f"   Best Performing Sentiment: {best_sentiment}")
    print(f"   Worst Performing Sentiment: {worst_sentiment}")
    print(f"   Highest Win Rate Sentiment: {sentiment_analysis['Win_Rate'].idxmax()}")
# 3. Trader Categories

print("\n3. TRADER SEGMENTATION")
print("-"*40)
for cat, count in trader_stats['Trader_Category'].value_counts().items():
    pct = count / len(trader_stats) * 100
    print(f"   {cat}: {count:,} traders ({pct:.1f}%)")
# 4. Top Coins

print("\n4. TOP TRADED ASSETS")
print("-"*40)
for i, (coin, row) in enumerate(coin_stats.head(5).iterrows(), 1):
    print(f"   {i}. {coin}: ${row['Total_Volume']:,.0f} volume, {row['Trade_Count']:,} trades")

print("\n" + "="*70)
# Strategic Recommendations

print("\n" + "="*70)
print("                 STRATEGIC RECOMMENDATIONS")
print("="*70)

recommendations = """

    SENTIMENT-BASED TRADING STRATEGY
         Consider contrarian strategies during extreme sentiment periods
         Increase position sizing confidence during historically profitable sentiment phases
         Implement sentiment-based risk management rules


    TIMING OPTIMIZATION
         Focus trading activity during hours with highest win rates
         Be cautious during typically losing time periods
         Consider day-of-week patterns for entry/exit timing


    RISK MANAGEMENT
         Reduce position sizes during extreme market sentiment
         Set tighter stop-losses when sentiment shifts rapidly
         Monitor leverage usage across different sentiment regimes


    TRADER DEVELOPMENT
         Study patterns of elite traders (top performers)
         Identify and avoid behaviors common among struggling traders
         Focus on consistency (win rate) over large individual gains


    ASSET SELECTION
         Diversify across multiple coins based on performance data
         Avoid over-concentration in high-volume but low-PnL assets
         Monitor correlation between asset performance and sentiment
        """


print(recommendations)


                    KEY FINDINGS SUMMARY

1. OVERALL TRADING STATISTICS
----------------------------------------
   Total Trades Analyzed: 211,224
   Unique Traders: 32
   Total Volume: 1,191,187,442.46
   Net PnL: 10,296,958.94
   Overall Win Rate: 83.2%

2. SENTIMENT IMPACT ON TRADING
----------------------------------------
   Best Performing Sentiment: Extreme Greed
   Worst Performing Sentiment: Neutral
   Highest Win Rate Sentiment: Extreme Greed

3. TRADER SEGMENTATION
----------------------------------------
   Elite: 28 traders (87.5%)
   At Risk: 4 traders (12.5%)

4. TOP TRADED ASSETS
----------------------------------------
   1. BTC: $644,232,117 volume, 26,064.0 trades
   2. HYPE: $141,990,206 volume, 68,005.0 trades
   3. SOL: $125,074,752 volume, 10,691.0 trades
   4. ETH: $118,280,994 volume, 11,158.0 trades
   5. @107: $55,760,859 volume, 29,992.0 trades


                 STRATEGIC RECOMMENDATIONS


    SENTIMENT-BASED TRADING STRATEGY
         Consider contrarian s

**10. Export Results**

In [48]:
# Save processed datasets

trader_stats.to_csv('csv_files/trader_performance_stats.csv')
coin_stats.to_csv('csv_files/coin_trading_stats.csv')
sentiment_analysis.to_csv('csv_files/sentiment_analysis.csv')
daily_metrics.to_csv('csv_files/daily_trading_metrics.csv')
# Save correlation matrix

correlations.to_csv('csv_files/sentiment_correlations.csv')

print("\nAll CSV files saved to csv_files/ directory:")
print("  - trader_performance_stats.csv")
print("  - coin_trading_stats.csv")
print("  - sentiment_analysis.csv")
print("  - daily_trading_metrics.csv")
print("  - sentiment_correlations.csv")

print("\nAll visualizations saved to outputs/ directory:")
print("  - top_coins_analysis.png")
print("  - trading_activity_patterns.png")
print("  - pnl_distribution.png")
print("  - trader_segmentation.png")
print("  - sentiment_overview.png")
print("  - behavior_by_sentiment.png")
print("  - sentiment_correlations.png")
print("  - time_patterns.png")
# Final Summary

print("\n" + "="*70)
print("                    ANALYSIS COMPLETE")
print("="*70)
print(f"\nTotal runtime artifacts:")
print(f"  - {len(df_trades):,} trades analyzed")
print(f"  - {len(trader_stats):,} trader profiles created")
print(f"  - 8 visualization files generated")
print(f"  - 5 CSV data exports created")
print("\nReady for report generation!")


All CSV files saved to csv_files/ directory:
  - trader_performance_stats.csv
  - coin_trading_stats.csv
  - sentiment_analysis.csv
  - daily_trading_metrics.csv
  - sentiment_correlations.csv

All visualizations saved to outputs/ directory:
  - top_coins_analysis.png
  - trading_activity_patterns.png
  - pnl_distribution.png
  - trader_segmentation.png
  - sentiment_overview.png
  - behavior_by_sentiment.png
  - sentiment_correlations.png
  - time_patterns.png

                    ANALYSIS COMPLETE

Total runtime artifacts:
  - 211,224 trades analyzed
  - 32 trader profiles created
  - 8 visualization files generated
  - 5 CSV data exports created

Ready for report generation!
